In [61]:
import pandas_access as mdb
from datetime import datetime, timedelta

import pandas as pd
import os
import btrdb

import urllib

from btrdb.utils import timez

In [11]:
db_path = "../data/epri_signatures/event_db.mdb"
# Listing the tables.
tables = mdb.list_tables(db_path)

In [13]:
# tbl = tables[0]
# print(tbl)
# df = mdb.read_table()
dfs = dict(zip(tables, [mdb.read_table(db_path, tbl) for tbl in tables]))

In [14]:
dfs.keys()

dict_keys(['Events', 'SiteFeeder', 'uniq_sitefeeder', 'SiteSource', 'Site', 'Table_sitenames_ids'])

In [23]:
dfs['Events'].sort_values('EventId')

,id,SiteName,FeederId,EventId,Phase,First Event,Last Event,Cause,Weather,Isolation Equipment Code,Failed Equipment Code,Details
170,187,Site0009,F_0000065,1,2,2005-01-01 10:10:34.487,2005-01-01 10:10:39.870,Tree,Clear Weather,Recloser,LINE CUT,Fault caused line recloser lockout. Tree Outsi...
171,188,Site0009,F_0000065,4,2,2005-01-01 10:10:34.487,2005-01-01 10:10:39.870,Tree,Clear Weather,Recloser,LINE CUT,Fault caused line recloser lockout. Tree Outsi...
172,189,Site0009,F_0000065,5,2,2005-01-01 10:10:34.487,2005-01-01 10:10:39.870,Tree,Clear Weather,Recloser,LINE CUT,Fault caused line recloser lockout. Tree Outsi...
173,190,Site0009,F_0000065,7,2,2005-01-01 10:10:34.487,2005-01-01 10:10:39.870,Tree,Clear Weather,Recloser,LINE CUT,Fault caused line recloser lockout. Tree Outsi...
174,191,Site0009,F_0000061,21,1,2005-01-07 19:20:10.599,2005-01-07 19:20:10.613,Equipment,Clear Weather,CO15,FUSE,Overhead Insulator Failure. BROKEN INSULATOR
...,...,...,...,...,...,...,...,...,...,...,...,...
287,310,Site0003,F_0000050,21877,4,2007-07-30 18:28:18.000,2007-07-30 18:28:18.000,NaN,NaN,NaN,NaN,t3527 10Amp tree limb.
259,280,Site0002,F_0000044,21878,1,2007-07-31 13:18:56.000,2007-07-31 13:18:56.000,NaN,NaN,NaN,NaN,last of the arching
282,305,Site0002,F_0000047,21879,1,2007-08-30 02:06:38.000,2007-08-30 02:06:38.000,NaN,NaN,NaN,NaN,t8937 100amp lightening strike.
283,306,Site0002,F_0000047,21880,1,2007-08-30 04:11:34.000,2007-08-30 04:11:34.000,NaN,NaN,NaN,NaN,pick up load after lightning strik


In [19]:
len(dfs['Site'])

92

In [25]:

res = urllib.request.urlopen('https://pqmon.epri.com/doe_folder/0001/0001.csv')


In [50]:
with urllib.request.urlopen('https://pqmon.epri.com/doe_folder/0001/0001.csv') as response:
   out = response.read().splitlines()

In [72]:
start = datetime.strptime(out[0].decode("utf-8").split(' - ')[1], "%m/%d/%Y %H:%M:%S.%f")

In [74]:
cols = out[2].decode("utf-8").split(', ')
cols[0] = 'time (s)'
cols[-1] = cols[-1]
print(cols)

['time (s)', 'Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic', 'In']


In [93]:
row0 = 3
df = pd.DataFrame(index=range(row0,len(out)-row0), columns=cols)
for i in df.index:
    newrow = pd.Series(out[i].decode("utf-8").split(', '),
                      index=cols).astype(float)
    
    df.loc[i] = newrow

In [94]:
df.head().dtypes

time (s)    object
Va          object
Vb          object
Vc          object
Ia          object
Ib          object
Ic          object
In          object
dtype: object

In [97]:
tstamp = (1e9*df['time (s)'] + timez.to_nanoseconds(start)).astype(int)

3     1104574234470333312
4     1104574234471374976
5     1104574234472416640
6     1104574234473458304
7     1104574234474499968
8     1104574234475541632
9     1104574234476583296
10    1104574234477624960
11    1104574234478666624
12    1104574234479708288
Name: time (s), dtype: int64

In [48]:
out[-10:]

[b'0.0729166666666665, 18576.240234375, -14831.1220703125, -4311.2666015625, 201.071990966797, -121.265060424805, -26.9477920532227, 52.8591384887695\r\n',
 b'0.0739583333333332, 19259.732421875, -8166.87060546875, -10861.2216796875, 215.582336425781, -74.6246566772461, -92.2443618774414, 48.7133178710938\r\n',
 b'0.0749999999999998, 17768.544921875, -633.964965820313, -16515.091796875, 200.035537719727, -36.2758750915527, -134.738952636719, 29.0207099914551\r\n',
 b'0.0760416666666665, 12935.3232421875, 6418.89501953125, -19334.40625, 134.738952636719, 29.0206985473633, -155.468032836914, 8.29161834716797\r\n',
 b'0.0770833333333332, 5906.8466796875, 12883.5087890625, -19084.478515625, 60.1143035888672, 83.9527359008789, -156.504486083984, -12.4374465942383\r\n',
 b'0.0781249999999998, -1646.63256835938, 17808.9296875, -16202.681640625, -1.03645348548889, 124.374420166016, -128.520233154297, -5.18226623535156\r\n',
 b'0.0791666666666665, -8441.9443359375, 19399.9375, -10420.798828125,